In [259]:
import json
import requests
import pandas as pd
import numpy as np
from pprint import pprint
from sqlalchemy import create_engine, text

In [260]:
version = "080823"

In [261]:
import psycopg2 # python -> psql connection
import psycopg2.extras
#Read db settings from local_settings.py
import CJDH_local_settings
dbname = CJDH_local_settings.local_settings['TRDL_PSQL_Analysis']['dbname']
user = CJDH_local_settings.local_settings['TRDL_PSQL_Analysis']['user']
password = CJDH_local_settings.local_settings['TRDL_PSQL_Analysis']['password']
host = "localhost"
port = 5432

connection_string = f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}'
engine = create_engine(connection_string)
engine.autocommit = True  # read documentation understanding when to Use & NOT use

## Player Summary DataFrame

In [262]:
#Player = Stats for each player for the season
headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
get=requests.get("https://fantasy.premierleague.com/api/bootstrap-static/", headers=headers, timeout=5)
data=json.loads(get.text)

In [263]:
from datetime import datetime
import pytz
import shutil

cols=list(data['elements'][0].keys())
player_raw = pd.DataFrame(data['elements'], columns = cols)

player_raw['now_datetime_UTC'] = datetime.now(tz=pytz.utc)

def get_gw(df, df_col:str):
    gw_id = df[df[df_col]==True]['id']
    if gw_id.empty==True:
        return int(0)
    else: 
        return int(gw_id.values)

eventdf = pd.DataFrame(data['events'])
player_raw['current_gw'] = get_gw(eventdf,df_col='is_current')
player_raw['next_gw'] = get_gw(eventdf,df_col='is_next')

print(list(player_raw.columns))
player = player_raw[['code','web_name','team','element_type','status','now_cost','transfers_in','transfers_in_event','transfers_out','transfers_out_event','event_points','goals_scored', 'assists', 'clean_sheets', 'goals_conceded','total_points','minutes','points_per_game',
'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat', 'ict_index', 'starts', 'expected_goals', 'expected_assists', 'expected_goal_involvements', 'expected_goals_conceded',
'influence_rank','creativity_rank','threat_rank','ict_index_rank','corners_and_indirect_freekicks_order','direct_freekicks_order','penalties_order',
'expected_goals_per_90', 'saves_per_90', 'expected_assists_per_90', 'expected_goal_involvements_per_90', 'expected_goals_conceded_per_90', 'goals_conceded_per_90', 'points_per_game_rank', 'selected_rank', 'starts_per_90', 'clean_sheets_per_90','now_datetime_UTC','current_gw','next_gw']]

['chance_of_playing_next_round', 'chance_of_playing_this_round', 'code', 'cost_change_event', 'cost_change_event_fall', 'cost_change_start', 'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next', 'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam', 'news', 'news_added', 'now_cost', 'photo', 'points_per_game', 'second_name', 'selected_by_percent', 'special', 'squad_number', 'status', 'team', 'team_code', 'total_points', 'transfers_in', 'transfers_in_event', 'transfers_out', 'transfers_out_event', 'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored', 'assists', 'clean_sheets', 'goals_conceded', 'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat', 'ict_index', 'starts', 'expected_goals', 'expected_assists', 'expected_goal_involvements', 'expected_goals_conceded', 'influence_rank', 'influence_rank_type', 'creativity_rank', 'creativity_rank_type', 'thre

In [264]:
def create_staging_table_player(cursor):
    cursor.execute(text("""
        DROP TABLE IF EXISTS player CASCADE;
        CREATE UNLOGGED TABLE player (
            code                    NUMERIC,
            web_name                TEXT,
            team                    TEXT,
            element_type            NUMERIC,
            status                  TEXT,
            now_cost                NUMERIC,
            transfers_in            NUMERIC,
            transfers_in_event      NUMERIC,
            transfers_out           NUMERIC,
            transfers_out_event     NUMERIC,
            event_points            NUMERIC,
            goals_scored            NUMERIC,
            assists                 NUMERIC,
            clean_sheets            NUMERIC,
            goals_conceded          NUMERIC,
            total_points            NUMERIC,
            minutes                 NUMERIC,
            points_per_game         NUMERIC,
            own_goals               NUMERIC,
            penalties_saved         NUMERIC,
            penalties_missed        NUMERIC,
            yellow_cards            NUMERIC,
            red_cards               NUMERIC,
            saves                   NUMERIC,
            bonus                   NUMERIC,
            bps                     NUMERIC,
            influence               NUMERIC,
            creativity              NUMERIC,
            threat                  NUMERIC,
            ict_index               NUMERIC,
            starts                  NUMERIC,
            expected_goals          NUMERIC,
            expected_assists        NUMERIC,
            expected_goal_involvements NUMERIC,
            expected_goals_conceded NUMERIC,
            influence_rank          NUMERIC,
            creativity_rank         NUMERIC,
            threat_rank             NUMERIC,
            ict_index_rank          NUMERIC,
            corners_and_indirect_freekicks_order NUMERIC,
            direct_freekicks_order  NUMERIC,
            penalties_order         NUMERIC,
            expected_goals_per_90   NUMERIC,
            saves_per_90            NUMERIC,
            expected_assists_per_90 NUMERIC,
            expected_goal_involvements_per_90 NUMERIC,
            expected_goals_conceded_per_90 NUMERIC,
            goals_conceded_per_90   NUMERIC,
            points_per_game_rank    NUMERIC,
            selected_rank           NUMERIC,
            starts_per_90           NUMERIC,
            clean_sheets_per_90     NUMERIC,
            now_datetime_UTC        TIMESTAMP,
            current_gw              NUMERIC,
            next_gw                 NUMERIC
        );"""))
# creating our schema  and sending the table to psql
with engine.connect() as cursor:
    create_staging_table_player(cursor)
    
player.to_sql('player', con=engine, if_exists='replace',index=False)

22

In [265]:
#Adding team names to playersummarydf (according to the team #)
teams_raw = pd.DataFrame(data['teams'])
print(teams_raw.columns)
teams = teams_raw[['code','id','name','strength','strength_overall_home', 'strength_overall_away',
       'strength_attack_home', 'strength_attack_away', 'strength_defence_home',
       'strength_defence_away']]


##

def create_staging_table_teams(cursor):
    cursor.execute(text("""
        DROP TABLE IF EXISTS player CASCADE;
        CREATE UNLOGGED TABLE player (
            code                    NUMERIC, -- permanent team code
            id                      NUMERIC, -- 23/24 season team code
            name                    TEXT,
            strength                NUMERIC,
            strength_overall_home   NUMERIC,
            strength_overall_away   NUMERIC,
            strength_attack_home    NUMERIC,
            strength_attack_away    NUMERIC,
            strength_defence_home   NUMERIC,
            strength_defence_away   NUMERIC
        );"""))
# creating our schema  and sending the table to psql
with engine.connect() as cursor:
    create_staging_table_teams(cursor)

teams.to_sql('teams', con=engine, if_exists='replace',index=False)

Index(['code', 'draw', 'form', 'id', 'loss', 'name', 'played', 'points',
       'position', 'short_name', 'strength', 'team_division', 'unavailable',
       'win', 'strength_overall_home', 'strength_overall_away',
       'strength_attack_home', 'strength_attack_away', 'strength_defence_home',
       'strength_defence_away', 'pulse_id'],
      dtype='object')


20

In [240]:
#Append Player Summaries to fixtures
get=requests.get("https://fantasy.premierleague.com/api/fixtures/", headers=headers, timeout=5)
data=json.loads(get.text)

fixturedf = pd.DataFrame(data)
home_team = fixturedf[['code', 'event', 'finished', 'finished_provisional', 'id','kickoff_time', 'minutes', 'provisional_start_time', 'started','team_a', 'team_a_score', 'team_h', 'team_h_score', 'stats','team_h_difficulty', 'team_a_difficulty']].copy()
home_team.rename(columns={'team_a':'opp_team', 'team_a_score':'opp_team_score', 'team_h':'team','team_h_score':'team_score','team_h_difficulty':'team_difficulty', 'team_a_difficulty':'opp_team_difficulty'}
                          ,inplace=True)
away_team = fixturedf[['code', 'event', 'finished', 'finished_provisional', 'id','kickoff_time', 'minutes', 'provisional_start_time', 'started', 'team_a', 'team_a_score', 'team_h', 'team_h_score', 'stats','team_h_difficulty', 'team_a_difficulty']].copy()
away_team.rename(columns={'team_a':'team', 'team_a_score':'team_score', 'team_h':'opp_team','team_h_score':'opp_team_score','team_h_difficulty':'opp_team_difficulty', 'team_a_difficulty':'team_difficulty'}
                          ,inplace=True)
fixtures = pd.concat([home_team, away_team])
print(fixtures.columns)
fixtures.head()

Index(['code', 'event', 'finished', 'finished_provisional', 'id',
       'kickoff_time', 'minutes', 'provisional_start_time', 'started',
       'opp_team', 'opp_team_score', 'team', 'team_score', 'stats',
       'team_difficulty', 'opp_team_difficulty'],
      dtype='object')


,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,opp_team,opp_team_score,team,team_score,stats,team_difficulty,opp_team_difficulty
0,2367552,NaN,False,False,15,None,0,False,None,6,None,12,None,[],2,2
1,2367713,NaN,False,False,176,None,0,False,None,4,None,13,None,[],3,5
2,2367538,1.0,False,False,1,2023-08-11T19:00:00Z,0,False,False,13,None,6,None,[],5,2
3,2367540,1.0,False,False,2,2023-08-12T11:30:00Z,0,False,False,16,None,1,None,[],2,4
4,2367539,1.0,False,False,3,2023-08-12T14:00:00Z,0,False,False,19,None,3,None,[],2,2


In [241]:
#Adding fixture information

def create_staging_table_fixtures(cursor):
    cursor.execute(text("""
        DROP TABLE IF EXISTS player CASCADE;
        CREATE UNLOGGED TABLE player (
            code                    NUMERIC,
            event                   NUMERIC,
            finished                BOOLEAN,
            finished_provisional    BOOLEAN,
            id                      NUMERIC,
            kickoff_time            TIMESTAMP,
            minutes                 NUMERIC,
            provisional_start_time  BOOLEAN,
            started                 BOOLEAN,
            opp_team                NUMERIC,
            opp_team_score          NUMERIC,
            team                    NUMERIC,
            team_score              NUMERIC,
            stats                   TEXT[],
            team_difficulty         NUMERIC,
            opp_team_difficulty     NUMERIC
        );"""))
# creating our schema  and sending the table to psql
with engine.connect() as cursor:
    create_staging_table_fixtures(cursor)

fixtures.to_sql('fixtures', con=engine, if_exists='replace',index=False)

760

In [242]:
query = """
        SELECT *
        FROM player
        LEFT JOIN teams
        ON player.team = teams.id
        ORDER BY total_points DESC
        ;
        """
with engine.connect() as conn:
    dat = pd.read_sql_query(query, conn)
print(dat.columns)
dat

Index(['code', 'web_name', 'team', 'element_type', 'status', 'now_cost',
       'transfers_in', 'transfers_in_event', 'transfers_out',
       'transfers_out_event', 'event_points', 'goals_scored', 'assists',
       'clean_sheets', 'goals_conceded', 'total_points', 'minutes',
       'points_per_game', 'own_goals', 'penalties_saved', 'penalties_missed',
       'yellow_cards', 'red_cards', 'saves', 'bonus', 'bps', 'influence',
       'creativity', 'threat', 'ict_index', 'starts', 'expected_goals',
       'expected_assists', 'expected_goal_involvements',
       'expected_goals_conceded', 'influence_rank', 'creativity_rank',
       'threat_rank', 'ict_index_rank', 'corners_and_indirect_freekicks_order',
       'direct_freekicks_order', 'penalties_order', 'expected_goals_per_90',
       'saves_per_90', 'expected_assists_per_90',
       'expected_goal_involvements_per_90', 'expected_goals_conceded_per_90',
       'goals_conceded_per_90', 'points_per_game_rank', 'selected_rank',
       'starts

,code,web_name,team,element_type,status,now_cost,transfers_in,transfers_in_event,transfers_out,transfers_out_event,...,code,id,name,strength,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away
0,223094,Haaland,13,4,a,140,0,0,0,0,...,43,13,Man City,5,1350,1355,1320,1340,1320,1370
1,78830,Kane,18,4,a,125,0,0,0,0,...,6,18,Spurs,3,1130,1200,1110,1200,1150,1200
2,118748,Salah,11,3,a,125,0,0,0,0,...,14,11,Liverpool,4,1220,1285,1170,1270,1260,1300
3,184029,Ødegaard,1,3,a,85,0,0,0,0,...,3,1,Arsenal,4,1230,1285,1250,1250,1210,1320
4,176297,Rashford,14,3,a,90,0,0,0,0,...,1,14,Man Utd,4,1145,1275,1150,1310,1140,1240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,477555,Bobb,13,3,a,45,0,0,0,0,...,43,13,Man City,5,1350,1355,1320,1340,1320,1370
612,192290,Roberts,6,2,a,45,0,0,0,0,...,90,6,Burnley,2,1060,1080,1060,1080,1060,1080
613,233425,Connolly,5,4,u,45,0,0,0,0,...,36,5,Brighton,3,1165,1210,1120,1200,1210,1240
614,168090,Dahoud,5,3,a,50,0,0,0,0,...,36,5,Brighton,3,1165,1210,1120,1200,1210,1240


In [251]:
query = """SELECT *,
                player.code as player_code,
                teams.id as team_id
            FROM player
            LEFT JOIN teams
            ON player.team = teams.id
        ;
        """
with engine.connect() as conn:
    currentgwplayerdf = pd.read_sql_query(query, conn)
print(currentgwplayerdf.columns)
currentgwplayerdf

Index(['code', 'web_name', 'team', 'element_type', 'status', 'now_cost',
       'transfers_in', 'transfers_in_event', 'transfers_out',
       'transfers_out_event', 'event_points', 'goals_scored', 'assists',
       'clean_sheets', 'goals_conceded', 'total_points', 'minutes',
       'points_per_game', 'own_goals', 'penalties_saved', 'penalties_missed',
       'yellow_cards', 'red_cards', 'saves', 'bonus', 'bps', 'influence',
       'creativity', 'threat', 'ict_index', 'starts', 'expected_goals',
       'expected_assists', 'expected_goal_involvements',
       'expected_goals_conceded', 'influence_rank', 'creativity_rank',
       'threat_rank', 'ict_index_rank', 'corners_and_indirect_freekicks_order',
       'direct_freekicks_order', 'penalties_order', 'expected_goals_per_90',
       'saves_per_90', 'expected_assists_per_90',
       'expected_goal_involvements_per_90', 'expected_goals_conceded_per_90',
       'goals_conceded_per_90', 'points_per_game_rank', 'selected_rank',
       'starts

,code,web_name,team,element_type,status,now_cost,transfers_in,transfers_in_event,transfers_out,transfers_out_event,...,name,strength,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,player_code,team_id
0,445122,J.Timber,1,2,a,50,0,0,0,0,...,Arsenal,4,1230,1285,1250,1250,1210,1320,445122,1
1,200641,Nelson,1,3,d,50,0,0,0,0,...,Arsenal,4,1230,1285,1250,1250,1210,1320,200641,1
2,204480,Rice,1,3,a,55,0,0,0,0,...,Arsenal,4,1230,1285,1250,1250,1210,1320,204480,1
3,206325,Zinchenko,1,2,a,50,0,0,0,0,...,Arsenal,4,1230,1285,1250,1250,1210,1320,206325,1
4,84450,Xhaka,1,3,u,60,0,0,0,0,...,Arsenal,4,1230,1285,1250,1250,1210,1320,84450,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,510363,Chiquinho,20,3,u,45,0,0,0,0,...,Wolves,3,1065,1100,1070,1160,1060,1080,510363,20
612,490721,Bueno,20,2,a,45,0,0,0,0,...,Wolves,3,1065,1100,1070,1160,1060,1080,490721,20
613,431960,Bolla,20,2,a,40,0,0,0,0,...,Wolves,3,1065,1100,1070,1160,1060,1080,431960,20
614,79602,Bentley,20,1,a,40,0,0,0,0,...,Wolves,3,1065,1100,1070,1160,1060,1080,79602,20


In [269]:
query = """SELECT *
            FROM teams
        ;
        """
with engine.connect() as conn:
    df = pd.read_sql_query(query, conn)
print(df.columns)

Index(['code', 'id', 'name', 'strength', 'strength_overall_home',
       'strength_overall_away', 'strength_attack_home', 'strength_attack_away',
       'strength_defence_home', 'strength_defence_away'],
      dtype='object')


In [276]:
query = """
        WITH players AS (
            SELECT *,
                player.code as player_code,
                teams.id as team_id
            FROM player
            LEFT JOIN teams
            ON player.team = teams.id
        )
        SELECT *
        FROM fixtures
        INNER JOIN players
            ON fixtures.team = players.team_id
                AND fixtures.event = players.current_gw
        INNER JOIN (
            SELECT
                code as opp_team_code,
                id as opp_team_id,
                name as opp_team_name,
                strength as opp_team_strength,
                strength_overall_home opp_team_strength_overall_home,
                strength_overall_away opp_team_strength_overall_away,
                strength_attack_home opp_team_strength_attack_home,
                strength_attack_away opp_team_strength_attack_away,
                strength_defence_home opp_team_strength_defence_home,
                strength_defence_away opp_team_strength_defence_away
            FROM teams) as opp_teams
            ON fixtures.opp_team = opp_teams.opp_team_id
        ORDER BY total_points DESC
        ;
        """
with engine.connect() as conn:
    df = pd.read_sql_query(query, conn)
print(df.columns)
df
# df.to_csv('fixture_df.csv')

Index(['code', 'event', 'finished', 'finished_provisional', 'id',
       'kickoff_time', 'minutes', 'provisional_start_time', 'started',
       'opp_team', 'opp_team_score', 'team', 'team_score', 'stats',
       'team_difficulty', 'opp_team_difficulty', 'code', 'web_name', 'team',
       'element_type', 'status', 'now_cost', 'transfers_in',
       'transfers_in_event', 'transfers_out', 'transfers_out_event',
       'event_points', 'goals_scored', 'assists', 'clean_sheets',
       'goals_conceded', 'total_points', 'minutes', 'points_per_game',
       'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards',
       'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity',
       'threat', 'ict_index', 'starts', 'expected_goals', 'expected_assists',
       'expected_goal_involvements', 'expected_goals_conceded',
       'influence_rank', 'creativity_rank', 'threat_rank', 'ict_index_rank',
       'corners_and_indirect_freekicks_order', 'direct_freekicks_order',
       'pena

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,opp_team,...,opp_team_code,opp_team_id,opp_team_name,opp_team_strength,opp_team_strength_overall_home,opp_team_strength_overall_away,opp_team_strength_attack_home,opp_team_strength_attack_away,opp_team_strength_defence_home,opp_team_strength_defence_away


In [255]:

query = """
        WITH players AS (
            SELECT *,
                player.code as player_code,
                teams.id as team_id
            FROM player
            LEFT JOIN teams
            ON player.team = teams.id
        )
        SELECT fixtures.team, players.team_id, players.web_name, fixtures.event, players.current_gw
        FROM fixtures
        INNER JOIN players
        ON fixtures.team = players.team_id
            AND fixtures.event = players.current_gw
        ORDER BY total_points DESC
        ;
        """
with engine.connect() as conn:
    currentgwplayerdf = pd.read_sql_query(query, conn)
print(currentgwplayerdf.columns)
currentgwplayerdf

Index(['team', 'team_id', 'web_name', 'event', 'current_gw'], dtype='object')


,team,team_id,web_name,event,current_gw


In [256]:
#AFTER GW1: append to existing csv if gw not already in csv
# file_name = "PlayerSummaryGWs.csv"
# new_file_name = f"PlayerSummaryGWs_{version}.csv"
# file_url = f"data/{file_name}"
# currentgwplayerdf.to_csv(file_url)
# df = pd.read_csv(file_url)
# gw = get_gw(eventdf,df_col='is_current')
# print(gw)
# print(set(df['current_gw']))
# if gw not in set(df['current_gw']): #if api current_gw not in the csv file, append new data
#     currentgwplayerdf.to_csv(file_url,mode='a', index=False, header=False)
#     shutil.copy(file_url,f"data/{new_file_name})
#     print("Data Appended!!")
# else:
#     print("Data Not Appended!! GW already exists in file")

In [ ]:
#Graph fixture difficulty for a player over time

In [ ]:
#Join all Player Detail stats for each Fixture

In [ ]:
# from datetime import datetime
# import pytz
# import shutil

# playersummarydf['now_datetime_UTC'] = datetime.now(tz=pytz.utc)

# def get_gw(df, df_col:str):
#     gw_id = df[df[df_col]==True]['id']
#     if gw_id.empty==True:
#         return int(0)
#     else: 
#         return int(gw_id.values)

# eventdf = pd.DataFrame(data['events'])
# playersummarydf['current_gw'] = get_gw(eventdf,df_col='is_current')
# playersummarydf['next_gw'] = get_gw(eventdf,df_col='is_next')
# playersummarydf['next_gw'].value_counts()

# playersummarydf=pd.merge(playersummarydf,eventdf,how='left',left_on='current_gw',right_on='id', suffixes=('_player',''))
# playersummarydf=pd.merge(playersummarydf,eventdf,how='left',left_on='next_gw',right_on='id',suffixes=('_current_gw','_next_gw'))

In [156]:
#Player details dataframe

def get_player_details():
    #Get a list of columns for player details dataframe
    playerdetailurl="https://fantasy.premierleague.com/api/element-summary/1/"
    get=requests.get(playerdetailurl)
    p_detail_data=json.loads(get.text)      
    try:
        cols=list(p_detail_data['history'][0].keys())
    except IndexError:
        cols=None

    #Get data for each player, for each gameweek, for each column
    bootstrapurl="https://fantasy.premierleague.com/api/bootstrap-static/"
    get=requests.get(bootstrapurl)
    bootstrapdata=json.loads(get.text)

    #Get mp, max player
    mpl=[] #max player list
    for i in range(0,len(bootstrapdata['elements'])):
        mpl.append(bootstrapdata['elements'][i].get('id'))
    mp=max(mpl)

    e=[] #exceptions 1
    e1=[] #exceptions 2

    playerdetails=[]

    for playerid in range(1,mp+1):  
        try:
            print(playerid)
            playerdetailurl="https://fantasy.premierleague.com/api/element-summary/"+str(playerid)+"/"
            get=requests.get(playerdetailurl)
            p_detail_data=json.loads(get.text)

            for gw in range(0,39):    
                try:
                    row=p_detail_data['history'][gw]
                    r=[]
                    for col in cols:
                        r.append(row.get(col))
                    playerdetails.append(r)
                except IndexError as ex:
                    e.extend([playerid, gw,ex])    
        except Exception as ex1:
            e1.extend([playerid,gw,ex1])
    return playerdetails,cols

playerdetails,cols=get_player_details()
            

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [157]:
#Add Player Details

# df = pd.merge(fixturesdf, playerdetailsdf,how='left',left_on=['team','event'],right_on=['team','current_gw'], suffixes=('','_player'))

In [158]:
#get Player Summary only for undrafted players!

# code_list = codes.split()
# for i in range(0, len(code_list)):
#     code_list[i] = int(code_list[i])
# code_list

# playersummarydf['code'].isin(set(code_list)).value_counts()
# playersummarydf[~playersummarydf['code'].isin(code_list)].sort_values(by='total_points',ascending=False)[['code','web_name','total_points']].head(10)
# playersummarydf = playersummarydf[~playersummarydf['code'].isin(code_list)]